# Konvertering av Bergens registeret



In [1]:
import polars as pl

In [2]:
'''
Medlemsnummer intervall fra 11000 til 16000.
Medlemsnummer i bergens registeret strekker seg mellom 1 og 6000
I Cornerstone er det ledige nummer intervall mellom 11000 og 16000
For å få de gamle medlemsnummer innenfor intervallet så plusser vi 11000 til gammelt medlemsnummer
Bergen hadde 282 medlemme ved årskiftet til 2025
'''
RANGE_START: int = 11000

In [3]:
# leser inn Data exportert i fra Bergens Access database
# I fra den tabellen som heter "tblInnmeldte"
df = (pl.read_excel("tblInnmeldte 2025-06-12.xlsx")
    # Vet ikke hvorfor dette blir gjort, men det er det som skjer i access databasen "qryMedlemmer" -
    # som er utgangspunkt for alle data som vises i Access
    # uten filter finnes 333 medlemmer, mens med filter finnes 284 medlemmer.
    # det er disse som har vært utgangspunkt for medlems opptellig
    .filter(pl.col("POSTNR").is_between(pl.lit("100"), pl.lit("9000"), closed="both"))
)
# Medlems sjekk, det er viktig at dette tallet er nokså likt som rapportert in til statsforvalteren 282
print("Antall medlemmer i Bergens registeret -->", df.filter(pl.col("Kategori").str.starts_with("M"))["MedlemID"])
print("Alle data i fra excel fil -->", df)

Could not determine dtype for column 43, falling back to string


Antall medlemmer i Bergens registeret --> shape: (284,)
Series: 'MedlemID' [i64]
[
	3570
	4536
	3800
	3720
	2073
	…
	3432
	27
	3294
	2451
	4638
]
Alle data i fra excel fil --> shape: (582, 46)
┌──────────┬──────────────┬─────────────┬──────────┬───┬─────────────┬──────────┬─────────┬────────┐
│ MedlemID ┆ Innmeldingsd ┆ Er den      ┆ Menighet ┆ … ┆ UtmeldtDato ┆ DødsDato ┆ Kommune ┆ KID-nr │
│ ---      ┆ ato          ┆ innmeldte   ┆ ---      ┆   ┆ ---         ┆ ---      ┆ ---     ┆ ---    │
│ i64      ┆ ---          ┆ døpt ?      ┆ str      ┆   ┆ date        ┆ str      ┆ str     ┆ str    │
│          ┆ date         ┆ ---         ┆          ┆   ┆             ┆          ┆         ┆        │
│          ┆              ┆ bool        ┆          ┆   ┆             ┆          ┆         ┆        │
╞══════════╪══════════════╪═════════════╪══════════╪═══╪═════════════╪══════════╪═════════╪════════╡
│ 2812     ┆ null         ┆ false       ┆ Bergen   ┆ … ┆ null        ┆ null     ┆ AURLAND ┆        │

In [4]:
# Oppdatering av medlemsID til Cornerstone standard og litt vasking også
df = df.with_columns(
    # Lager nye medlemsnummere som er tilpasset Cornerstone
    (pl.col("MedlemID") + RANGE_START).alias("MedlemID"),
    (pl.col("SamID") + RANGE_START).alias("SamID"),
    (pl.col("MorID") + RANGE_START).alias("MorID"),
    (pl.col("FarID") + RANGE_START).alias("FarID"),
    # Tar bort Personnummer der det bare er skervet in fødselsdato delen
    # Har sjekket at "Fødselsdato" er satt på alle det gjelder
    pl.when(pl.col("Personnummer").str.len_chars() == 6)
      .then(None)
      .otherwise(pl.col("Personnummer"))
      .alias("Personnummer")
)


In [5]:
df.select("MedlemID", "SamID", "FORNAVN", "EFTERNAVN", "Personnummer"
         ).sort("SamID"
               )#.filter(pl.col("Personnummer").str.len_chars() == 6)

MedlemID,SamID,FORNAVN,EFTERNAVN,Personnummer
i64,i64,str,str,str
14571,null,"""Bendik Olai""","""Agdal""","""28029245543"""
14570,null,"""Oda Johanna""","""Agdal""","""27069744422"""
15535,null,"""Rita""","""Agdal""",""""""
14642,null,"""Sissel""","""Aggerholm""",""""""
15493,null,"""Johan""","""Ahlbom""",""""""
…,…,…,…,…
15547,15604,"""Tone Bell""","""Rysst""","""22055032878"""
15551,15606,"""Tora Døssland""","""Eikeland""","""16065247058"""
13248,15628,"""Katarina""","""Lunde""","""03089142074"""


## Rekkefølge på konverteringen
- Alle som har fødselsnummer, medlemmer med flere
- Alle som er registrert med "MedlemID", "SamID", "MorID", "FarID"
- Alle som har "MedlemID" i "SamID", "MorID", "FarID"
- 
- Det vil være overlappende "MedlemID" for de som er listet over. Få ut en liste og krymp den til disticte verdier av "MedlemID"
- Resterende personer vil ha datakvalitet av varierende kvalitet. Disse konverteres til slutt. Det er resterende som ikke er med i den oppbyggde "MedlemID" listen. Ingen av disse er medlemmer. De har heller ikke forbindelser til andre personer.

### Denne er ikke nødvendig, så lenge vi gjør den siste testen ved henting av data fra excel filen
``` python
df_medlemId = (
    df
    .filter(
        (pl.col("Kategori").str.starts_with("M")) &
        # Vet ikke hvorfor dette blir gjort, men det er det som skjer i access databasen "qryMedlemmer"
        (pl.col("POSTNR").is_between(pl.lit("100"), pl.lit("9000"), closed="both"))
    )
    .select("MedlemID")
)
df_medlemId
```

In [6]:
# Henter ut alle personer som har andre personer knyttet til seg
# Fornavn og etternavn hentes også på personen som har tilknyttninger
df_connectedMedlemer = pl.concat([
    df.filter(pl.col(c).is_not_null())
      .select(
          pl.col("FORNAVN"),
          pl.col("EFTERNAVN"),
          pl.lit(c).alias("type"),
          pl.col("MedlemID"),
          pl.col(c).alias("ConnectedMedlemID")
      )
    for c in ["SamID", "MorID", "FarID"]
])
print("Personer som er knyttet til andre personer mor, far, sambo -->", df_connectedMedlemer)

Personer som er knyttet til andre personer mor, far, sambo --> shape: (436, 5)
┌───────────────────────┬─────────────┬───────┬──────────┬───────────────────┐
│ FORNAVN               ┆ EFTERNAVN   ┆ type  ┆ MedlemID ┆ ConnectedMedlemID │
│ ---                   ┆ ---         ┆ ---   ┆ ---      ┆ ---               │
│ str                   ┆ str         ┆ str   ┆ i64      ┆ i64               │
╞═══════════════════════╪═════════════╪═══════╪══════════╪═══════════════════╡
│ Martin Daniel Peter   ┆ Aeschlimann ┆ SamID ┆ 13812    ┆ 13617             │
│ Terje                 ┆ Alræk       ┆ SamID ┆ 13803    ┆ 12705             │
│ Gry Veronica          ┆ Alsos       ┆ SamID ┆ 14800    ┆ 12729             │
│ Vahan                 ┆ Babayan     ┆ SamID ┆ 14792    ┆ 14631             │
│ Jorun                 ┆ Barane      ┆ SamID ┆ 15537    ┆ 14463             │
│ …                     ┆ …           ┆ …     ┆ …        ┆ …                 │
│ Kristoffer Nikolai    ┆ Wüthrich    ┆ FarID ┆ 1251

In [7]:
# Finner alle ID'er som ikke eksisterer.
# Her er det feil i dato og dette er en del av vasken

# 1. Anti-join: IDs in connected that aren't in df
missing_ids = df_connectedMedlemer.join(
    df, 
    left_on="ConnectedMedlemID",      # Key in the left DataFrame
    right_on="MedlemID",              # Key in the right DataFrame
    how="anti"
).select(pl.col("ConnectedMedlemID").unique().alias("MedlemID"))
print("missing ids -->", missing_ids)

# Sletter alle ID'er som ikke finnes fra vår Connected liste
# Sletter alle koblinger til personer som ikke finnes i registeret
df_connectedMedlemer = df_connectedMedlemer.join(
    missing_ids,
    left_on="ConnectedMedlemID",      # Key in the left DataFrame
    right_on="MedlemID",              # Key in the right DataFrame
    how="anti"
)
print("Liste over samenkoblede personer, etter at ugyldige er slettet -->", df_connectedMedlemer)

missing ids --> shape: (35, 1)
┌──────────┐
│ MedlemID │
│ ---      │
│ i64      │
╞══════════╡
│ 11025    │
│ 11112    │
│ 11113    │
│ 11212    │
│ 11215    │
│ …        │
│ 14743    │
│ 15444    │
│ 15445    │
│ 15485    │
│ 15533    │
└──────────┘
Liste over samenkoblede personer, etter at ugyldige er slettet --> shape: (380, 5)
┌───────────────────────┬─────────────┬───────┬──────────┬───────────────────┐
│ FORNAVN               ┆ EFTERNAVN   ┆ type  ┆ MedlemID ┆ ConnectedMedlemID │
│ ---                   ┆ ---         ┆ ---   ┆ ---      ┆ ---               │
│ str                   ┆ str         ┆ str   ┆ i64      ┆ i64               │
╞═══════════════════════╪═════════════╪═══════╪══════════╪═══════════════════╡
│ Martin Daniel Peter   ┆ Aeschlimann ┆ SamID ┆ 13812    ┆ 13617             │
│ Terje                 ┆ Alræk       ┆ SamID ┆ 13803    ┆ 12705             │
│ Gry Veronica          ┆ Alsos       ┆ SamID ┆ 14800    ┆ 12729             │
│ Vahan                 ┆ Babayan

In [8]:

# Tror ikke dette blir nødvendig. Personers forbindelser blir oppdatert etter at alle
# relevante personer er lagt inn i egen ferdig vasket tabell under "df_connectedMedlemer"

# Oppdaterer df og setter alle ikke eksisterende kontakter til null
#df = df.with_columns([
#    pl.when(pl.col(c).is_in(missing_ids["MedlemID"].to_list()))
#      .then(None)
#      .otherwise(pl.col(c))
#      .alias(c)
#    for c in ["SamID", "MorID", "FarID"]
#])


In [9]:
# Henter inn fornavn og etternavn til den personen som er tilknyttet
# Denne listen benyttes til å koble alle personer i Cornerstone
# etter at alle personer er lagt inn.

# Perform the join by preparing the right side first
df_connectedMedlemer = df_connectedMedlemer.join(
    # 1. Select and rename columns from 'df' on the fly
    df.select(
        pl.col("MedlemID"),  # The join key must be included
        pl.col("FORNAVN").alias("Connected_FORNAVN"),
        pl.col("EFTERNAVN").alias("Connected_EFTERNAVN")
    ),
    left_on="ConnectedMedlemID",
    right_on="MedlemID",
    how="left"  # Using 'left' is safer to not lose rows from the left table
)
print("Liste over samenkoblede personer, brukes til konvertering av relasjoner -->", df_connectedMedlemer)

Liste over samenkoblede personer, brukes til konvertering av relasjoner --> shape: (380, 7)
┌────────────────┬─────────────┬───────┬──────────┬────────────────┬───────────────┬───────────────┐
│ FORNAVN        ┆ EFTERNAVN   ┆ type  ┆ MedlemID ┆ ConnectedMedle ┆ Connected_FOR ┆ Connected_EFT │
│ ---            ┆ ---         ┆ ---   ┆ ---      ┆ mID            ┆ NAVN          ┆ ERNAVN        │
│ str            ┆ str         ┆ str   ┆ i64      ┆ ---            ┆ ---           ┆ ---           │
│                ┆             ┆       ┆          ┆ i64            ┆ str           ┆ str           │
╞════════════════╪═════════════╪═══════╪══════════╪════════════════╪═══════════════╪═══════════════╡
│ Martin Daniel  ┆ Aeschlimann ┆ SamID ┆ 13812    ┆ 13617          ┆ Drude         ┆ Isene         │
│ Peter          ┆             ┆       ┆          ┆                ┆               ┆               │
│ Terje          ┆ Alræk       ┆ SamID ┆ 13803    ┆ 12705          ┆ Torild        ┆ Alræk         │

In [10]:
# Samler sammen personer som vi antar har kavalitets data
# Til den første runden av konvertering
df_kvalitetsdataID = pl.concat([
    # Alle personer som har kontakter på seg
    df_connectedMedlemer["MedlemID"],
    # Alle personer som er en kontakt på en annen person
    df_connectedMedlemer["ConnectedMedlemID"],
    # Alle personer som har status som medlem
    df.filter(pl.col("Kategori").str.starts_with("M"))["MedlemID"]
]).unique().to_frame("MedlemID")
print("Liste over personers ID som vi antar har høy data kvalitet -->", df_kvalitetsdataID)

Liste over personers ID som vi antar har høy data kvalitet --> shape: (407, 1)
┌──────────┐
│ MedlemID │
│ ---      │
│ i64      │
╞══════════╡
│ 11018    │
│ 11027    │
│ 11084    │
│ 11208    │
│ 11209    │
│ …        │
│ 15638    │
│ 15649    │
│ 15650    │
│ 15659    │
│ 15663    │
└──────────┘


In [12]:
# Lager en liste av personer som har god kvalitet på data
# One‑liner using a semi‑join (no need for unique() – keeps only df rows whose MedlemID is in the concatenated list)
df_kvalitetsData = df.join(
    df_kvalitetsdataID,
    on="MedlemID"
)
print("Liste over personer som vi antar har høy data kvalitet -->",df_kvalitetsData)


# Lager en liste av personer som sjanse for dårligere kvalitet på data
# Option 1: Anti‑join (most idiomatic & performant)
df_looserData = df.join(
    df_kvalitetsData,
    on="MedlemID",
    how="anti"
)
print("Liste over personer som vi antar har dårlig data kvalitet -->", df_looserData)

Liste over personer som vi antar har høy data kvalitet --> shape: (407, 46)
┌──────────┬────────────┬────────────┬──────────┬───┬───────────┬──────────┬───────────┬───────────┐
│ MedlemID ┆ Innmelding ┆ Er den     ┆ Menighet ┆ … ┆ UtmeldtDa ┆ DødsDato ┆ Kommune   ┆ KID-nr    │
│ ---      ┆ sdato      ┆ innmeldte  ┆ ---      ┆   ┆ to        ┆ ---      ┆ ---       ┆ ---       │
│ i64      ┆ ---        ┆ døpt ?     ┆ str      ┆   ┆ ---       ┆ str      ┆ str       ┆ str       │
│          ┆ date       ┆ ---        ┆          ┆   ┆ date      ┆          ┆           ┆           │
│          ┆            ┆ bool       ┆          ┆   ┆           ┆          ┆           ┆           │
╞══════════╪════════════╪════════════╪══════════╪═══╪═══════════╪══════════╪═══════════╪═══════════╡
│ 13812    ┆ null       ┆ false      ┆ Bergen   ┆ … ┆ null      ┆ null     ┆ AURLAND   ┆           │
│ 14571    ┆ 2002-09-30 ┆ false      ┆ Bergen   ┆ … ┆ 2015-08-1 ┆ null     ┆           ┆           │
│          ┆   